In [10]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Flatten, Lambda, Dropout
import tensorflow.keras.backend as K

import numpy as np

In [2]:
class SimpleDense(Layer):
    
    def __init__(self, units=32):
        super(SimpleDense, self).__init__()
        self.units = units
        
    #We use def build(self, input_shape): to create the state of the layers and specify local input states.
    def build(self, input_shape): # Create the state of the layer (weights)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel',
                            initial_value=w_init(shape=(input_shape[-1], self.units),
                                                dtype='float32'),
                            trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias',
                            initial_value=b_init(shape=(self.units,), dtype='float32'),
                            trainable=True)
    
    def call(self, inputs): # Defines the computation from inputs to outputs
        return tf.matmul(inputs, self.w) + self.b
        
my_dense = SimpleDense(units=1)
x = tf.ones((1, 1))
y = my_dense(x)

print(my_dense.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.03313166]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


$ 2x - 1$

In [3]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys =  np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [4]:
my_layer = SimpleDense(1)

model = tf.keras.models.Sequential([my_layer])
model.compile(optimizer='sgd',
                loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))
print(my_layer.variables)

[[18.9814]]
[<tf.Variable 'simple_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973042]], dtype=float32)>, <tf.Variable 'simple_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99164236], dtype=float32)>]


In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

custom_layer = SimpleDense(128)

model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    custom_layer,
    Lambda(lambda x: tf.abs(x)),
    Dropout(0.1),
    Dense(units=10, activation='softmax')
])
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4323 - accuracy: 0.8756
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1209 - accuracy: 0.9640
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0892 - accuracy: 0.9716
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0724 - accuracy: 0.9777
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.0730 - accuracy: 0.9787


[0.07302159070968628, 0.9786999821662903]

In [6]:
print(custom_layer.variables)

[<tf.Variable 'simple_dense_2/kernel:0' shape=(784, 128) dtype=float32, numpy=
array([[-0.01821556, -0.06078629,  0.03296904, ..., -0.03211952,
        -0.00411339,  0.01171576],
       [-0.09093087, -0.0100612 ,  0.08894789, ...,  0.02703124,
         0.08309151,  0.00784083],
       [-0.10543244,  0.01748854, -0.04546477, ...,  0.01973497,
         0.04790882,  0.00461827],
       ...,
       [-0.05366775,  0.00759535, -0.0601759 , ..., -0.07406662,
        -0.05222664, -0.02748975],
       [ 0.02620558, -0.00838865, -0.03226786, ...,  0.00813237,
        -0.03600022,  0.04739312],
       [-0.07122918, -0.00015546,  0.0477097 , ..., -0.08269745,
        -0.02607518,  0.08252136]], dtype=float32)>, <tf.Variable 'simple_dense_2/bias:0' shape=(128,) dtype=float32, numpy=
array([-1.40060455e-01,  1.15310319e-01, -5.81937358e-02, -4.39076722e-02,
       -9.18452516e-02, -1.29172310e-01, -1.29385535e-02, -1.10927880e-01,
        6.23607077e-03,  7.23845288e-02,  2.28855424e-02, -3.78958769

In [7]:
class SimpleDenseWithActivation(Layer):
    # add an activation parameter
    def __init__(self, units=32, activation=None):
        super(SimpleDenseWithActivation, self).__init__() #The super keyword is used for inheriting functionalities from Keras’s Layer class.
        self.units = units
        
        # define the activation to get from the built-in activation layers in Keras
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel',
                            initial_value=w_init(shape=(input_shape[-1], self.units),
                                                dtype='float32'),
                            trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias',
                            initial_value=b_init(shape=(self.units,),
                                                dtype='float32'),
                            trainable=True)
        super().build(input_shape) #The super keyword is used for inheriting functionalities from Keras’s Layer class.
        
    def call(self, inputs): # Defines the computation from inputs to outputs --> call during training
        # pass the computation to the activation layer
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [8]:
model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    SimpleDenseWithActivation(128, activation='relu'),
    Dropout(0.1),
    Dense(units=10, activation='softmax')
])
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4589 - accuracy: 0.8686
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1314 - accuracy: 0.9618
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0898 - accuracy: 0.9729
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0703 - accuracy: 0.9788
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.0655 - accuracy: 0.9799


[0.06553016602993011, 0.9799000024795532]

#### Quadratic Layer

$ax^2 + bx + c$.

In [11]:
class SimpleQuadratic(Layer):
    def __init__(self, units=32, activation=None):
        super(SimpleQuadratic, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):  
        a_init = tf.random_normal_initializer()
        self.a = tf.Variable(name='a_kernel', 
                            initial_value=a_init(shape=(input_shape[-1], self.units),
                                                  dtype='float32'),
                            trainable=True)
        
        b_init = tf.random_normal_initializer()
        self.b = tf.Variable(name='b_kernel',
                            initial_value=b_init(shape=(input_shape[-1], self.units),
                                                dtype='float32'),
                            trainable=True)
        
        c_init = tf.zeros_initializer()
        self.c = tf.Variable(name='bias',
                            initial_value=c_init(shape=(self.units,),
                                                dtype='float32'),
                            trainable=True)
        super().build(input_shape)
        
    def call(self, inputs):
        x_squared = K.square(inputs)
        return self.activation(tf.matmul(x_squared, self.a) + tf.matmul(inputs, self.b) + self.c)

In [13]:
model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    SimpleQuadratic(128, activation='relu'),
    Dropout(0.1),
    Dense(units=10, activation='softmax')
])
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3952 - accuracy: 0.8833
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1246 - accuracy: 0.9626
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0854 - accuracy: 0.9744
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0642 - accuracy: 0.9794
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.0729 - accuracy: 0.9795


[0.07285595685243607, 0.9794999957084656]